In [1]:
from code_snippet_dataset import CodeSnippetDataset

ds = CodeSnippetDataset(languages=["Python"])

ModuleNotFoundError: ignored

In [ ]:
target_function = "def hello():\n   print('hello world!')"
dicty = dict();
dicty["input"] = []
dicty["target"] = []
counter = 0
for example in ds.dataset:
    if len(example["functions"]) > 0:
        dicty["input"].append(example["functions"][0])
        dicty["target"].append(target_function)
        counter += 1
    if counter == 100000:
        break

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(dicty)

In [ ]:
df.to_csv("dummy.csv")

In [1]:
!pip install datasets transformers evaluate
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-large")

prefix = "refine: "

def preprocess_function(examples):
    inputs = [prefix + example for example in examples["input"]]
    outputs = [example for example in examples["target"]]
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=128, truncation=True)
    return model_inputs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import Dataset
import pandas as pd
df = pd.read_csv("dummy.csv")
min_ds = Dataset.from_pandas(df)
tokenized_code = min_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [3]:
tokenized_code = tokenized_code.train_test_split(test_size=0.2)

In [4]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import evaluate
sacrebleu = evaluate.load("sacrebleu")

In [6]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [7]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!nvidia-smi

Mon Feb 27 20:22:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
training_args = Seq2SeqTrainingArguments(
    output_dir="dummy_code_to_code_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_code["train"],
    eval_dataset=tokenized_code["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/dummy_code_to_code_model is already a clone of https://huggingface.co/michaelnath/dummy_code_to_code_model. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target, input, Unnamed: 0. If target, input, Unnamed: 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 80000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16

Epoch,Training Loss,Validation Loss


Saving model checkpoint to dummy_code_to_code_model/checkpoint-500
Configuration saved in dummy_code_to_code_model/checkpoint-500/config.json
Configuration saved in dummy_code_to_code_model/checkpoint-500/generation_config.json
Model weights saved in dummy_code_to_code_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in dummy_code_to_code_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in dummy_code_to_code_model/checkpoint-500/special_tokens_map.json
tokenizer config file saved in dummy_code_to_code_model/tokenizer_config.json
Special tokens file saved in dummy_code_to_code_model/special_tokens_map.json
Saving model checkpoint to dummy_code_to_code_model/checkpoint-1000
Configuration saved in dummy_code_to_code_model/checkpoint-1000/config.json
Configuration saved in dummy_code_to_code_model/checkpoint-1000/generation_config.json
Model weights saved in dummy_code_to_code_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in dummy